*This notebook is under construction; please see the [pipeline](../getting_started/2_Pipeline.ipynb) and [nyc_taxi](https://examples.pyviz.org/nyc_taxi/nyc_taxi.html) notebooks for extensive examples of working with points.  For now, this section only includes information about spatially indexed datasets.*

# Spatial indexing

In most cases, Datashader must iterate through your entire dataset to render any plot, because it cannot assume the datapoints have been sorted in any particular order. Thus, the aggregation performance is dependent on the number of datapoints in your entire dataframe, not just those in the current viewport (x and y range). If you have a large dataset covering a wide area and you want to support fast local operations (e.g. if you have data at a global level but analysis is typically done in small local regions), you may want to use [spatialpandas](https://github.com/holoviz/spatialpandas), which is designed to work with Datashader and supports spatial indexing using [Hilbert curves](https://en.wikipedia.org/wiki/Hilbert_curve). Please see the [spatialpandas Overview](https://github.com/holoviz/spatialpandas/blob/main/examples/Overview.ipynb) notebook.

